In [1]:
import matplotlib.pyplot as plt
import calibration
import matplotlib.image as mpimg
import imagefilter
import glob
import warp
import findlane
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from line import Line
import drawlane
%matplotlib inline

In [ ]:
def draw(img, mtx, origin):
    leftfit, rightfit, leftx, rightx, lefty, righty = findlane.find_lane(img)
    ploty = findlane.get_ploty()
    return drawlane.lane_image(img, origin,mtx, ploty, leftfit,rightfit)

In [ ]:
def process(img, cali_dict=None, warp_m=None, in_m=None):
    if cali_dict is None:
        cali_dict = calibration.cam_cali()
    cali_img = calibration.cali_img(img, cali_dict)
    if warp_m is None:
        warp_src, warp_dst = warp.warp_points()
        warp_m, in_m = warp.warp_ms(warp_src,warp_dst)
    warped_img = warp.warp_img(cali_img, warp_m)
    filtered_img = imagefilter.color_gradient_filter(warped_img)
    return filtered_img


In [ ]:
images =[mpimg.imread(img) for img in glob.glob("test_images/test*.jpg")]

In [ ]:
cali_dict = calibration.cam_cali()
cali_images = [calibration.cali_img(img, cali_dict) for img in images]

In [ ]:
def plot_twin(images1, images2):
    for i in range(0, len(images1)):
        plt.figure()
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,9))
        ax1.imshow(images1[i])
        ax2.imshow(images2[i])

# plot_twin(images, cali_images)

In [ ]:
filtered_images = [imagefilter.color_gradient_filter(img) for img in cali_images]
#plot_twin(cali_images, filtered_images)

In [ ]:
src,dst = warp.warp_points()
m,in_m = warp.warp_ms(src, dst)

In [ ]:
warped_images = [warp.warp_img(img,m) for img in cali_images]
filtered_warped_images = [imagefilter.color_gradient_filter(img) for img in warped_images]
# plot_twin(warped_images, filtered_warped_images)


In [ ]:
final_images = [draw(filtered_warped_images[i],in_m, images[i]) for i in range(0, len(images))]
#plot_twin(filtered_warped_images, final_images)

In [ ]:
processed_images = [process(img) for img in images]
#plot_twin(images, processed_images)

In [ ]:
line1 = Line()
line1.update(processed_images[0])

In [ ]:
out_image = drawlane.lane_image(processed_images[0], images[0],in_m, line1.ploty, line1.left_best_fit,line1.right_best_fit)
#plt.imshow(out_image)

In [2]:
def process_video(video_file):
    lane_line = Line()
    cali_dict = calibration.cam_cali()
    warp_src, warp_dst = warp.warp_points()
    warp_m, in_m = warp.warp_ms(warp_src,warp_dst)

    def process_full(img):
        cali_img = calibration.cali_img(img, cali_dict)
        warped_img = warp.warp_img(cali_img, warp_m)
        filtered_img = imagefilter.color_gradient_filter(warped_img)
        lane_line.update(filtered_img)
        output = drawlane.lane_image(filtered_img, img,in_m, lane_line.ploty, lane_line.left_best_fit,lane_line.right_best_fit)
        return output
    clip = VideoFileClip(video_file)
    return clip.fl_image(process_full) #NOTE: this function expects color images!!

In [ ]:
white_output = 'output_1.mp4'
white_clip = process_video("project_video.mp4")
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video output_1.mp4
[MoviePy] Writing video output_1.mp4


 81%|████████  | 1018/1261 [01:58<00:32,  7.40it/s]

In [ ]:
white_output = 'output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))